In [1]:
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import tensorflow.keras.optimizers
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
import os
from tensorflow.keras.regularizers import l2

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread, imshow, imsave
import cv2
import numpy as np
import tensorflow
from sklearn.metrics import plot_confusion_matrix# This Python 3 environment comes with many helpful analytics libraries installed

In [2]:
def load(file_path):
    image = imread(file_path)
    return image
def display(image,title="Image"):
    plt.figure(figsize=[10,10])
    channels=len(image.shape)
    if channels<3:
        plt.imshow(image,cmap='Greys_r');
    else:
        plt.imshow(image);      
    plt.title(title);plt.axis("off"); 
    
def check_label(file_path):
    if os.path.getsize(file_path) == 0:
        return 0
    else :
        return 1
def load_data(directory):
    my_counter_0=0
    my_counter_1=0
    sequence=["x/train","x/test","x/valid"]
    train_data=[]
    test_data=[]
    valid_data=[]
    
    valid_labels=[]
    test_labels=[]    
    train_labels=[]
    for x in sequence:
        directory_search=directory+x[1:]+'/images'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                my_image=load(f)
                resized = cv2.resize(my_image,(240,240), interpolation = cv2.INTER_AREA)
                if x == 'x/train' :
                    train_data.append(resized)
                elif x == 'x/test' :
                    test_data.append(resized)
                else:
                    valid_data.append(resized)
    
        directory_search=directory+x[1:]+'/labels'
        for filename in os.listdir(directory_search):
            f = os.path.join(directory_search, filename)
            if os.path.isfile(f):
                label=check_label(f)
                if label==0:
                    my_counter_0+=1
                elif label==1:
                    my_counter_1+=1
                if x == 'x/train' :
                    train_labels.append(label)
                elif x == 'x/test' :
                    test_labels.append(label)
                else:
                    valid_labels.append(label)
    print("Defected Pictures : ",my_counter_1)
    print("Non-Defected Pictures : ",my_counter_0)

    return train_data,test_data,valid_data,train_labels,test_labels,valid_labels

In [3]:
train_data,test_data,valid_data,train_labels,test_labels,valid_labels=load_data("/kaggle/input/dataset-final/Defect_Detection_FinalVersion")
np_train_data=np.array(train_data)
np_train_data = np_train_data.reshape((np_train_data.shape[0], 240,240, 3))
np_train_data = np_train_data.astype("float32") / 255.0
np_train_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))

np_test_data=np.array(test_data)
np_test_data = np_test_data.reshape((np_test_data.shape[0], 240,240, 3))
np_test_data = np_test_data.astype("float32") / 255.0
np_test_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))

np_valid_data=np.array(valid_data)
np_valid_data = np_valid_data.reshape((np_valid_data.shape[0], 240,240, 3))
np_valid_data = np_valid_data.astype("float32") / 255.0
np_valid_labels = np.asarray(valid_labels).astype('float32').reshape((-1,1))

Defected Pictures :  993
Non-Defected Pictures :  1109


In [4]:
class VGG_19:    

    def build(imgRows, imgCols,numChannels,numClasses):
        vgg = Sequential()
        pretrained_vgg= tf.keras.applications.VGG19(include_top=False,input_shape=(240,240,3),pooling='avg',classes=2,weights='imagenet',classifier_activation="relu")
        for each_layer in pretrained_vgg.layers:
                each_layer.trainable=False

        vgg.add(pretrained_vgg)
        vgg.add(Dense(512, activation='relu'))
        vgg.add(Dense(256, activation='relu'))
        vgg.add(Dense(1,activation='relu'))
   
        return vgg

In [ ]:
model = VGG_19.build(numChannels=3, imgRows=640, imgCols=640,numClasses=2)
opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
model.fit(np_train_data, np_train_labels, batch_size=64, epochs=40,verbose=1,validation_data=(np_valid_data, np_valid_labels))

2023-01-08 15:53:02.033684: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


80150528/80134624 [==============================] - 0s 0us/step


2023-01-08 15:53:04.623503: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/40
29/29 [==============================] - 695s 24s/step - loss: 7.9139 - accuracy: 0.4682 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 2/40
29/29 [==============================] - 697s 24s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 3/40
29/29 [==============================] - 697s 24s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 4/40
29/29 [==============================] - 696s 24s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 5/40
29/29 [==============================] - 698s 24s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 6/40
29/29 [==============================] - 698s 24s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 7/40
29/29 [==============================] - 709s 25s/step - loss: 8.1764 - accuracy: 0.4638 - val_loss: 5.6900 - val_accuracy: 0.6269
Epoch 

In [ ]:
scores = model.evaluate(np_test_data, np_test_labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('VGG_19')